<div class="alert alert-block alert-info">
    
<center> 
    
# __Lead-Lag Portfolios__ 
### __Problem Description and Methodology__ 
    
</center>

</div>

## __1. Introduction__

**Time series A is said to lead time series B if A’s past values are more strongly associated with B’s future values than A’s future values are with B’s past values**. In multivariate time series systems, it has been observed that certain groups of variables partially lead the evolution of the system, while other variables follow this evolution with a time delay. The result is a lead–lag structure among the time series variables. One of the commonly studied problems in the financial lead–lag literature is to identify financial instruments that exhibit stronger tendencies to lead other instruments in terms of future returns, as changes in prices of some instruments tend to lead those of others.

Detecting lead-lag relationships among a large set of financial instruments is a complex task. Existing literature often relies on ad-hoc methods to select leaders and followers, using these sets of assets in investment strategies to assess the economic significance of lead-lag relationships. These studies explicitly assume a connection between instrument characteristics and lead-lag relationships in their proposed methodologies. In this repository, we aim to implement a methodology for identifying lead-lag relationships without explicitly assuming a link between instrument characteristics and lead-lag dynamics. Instead, our focus is on a data-driven approach that utilizes instrument returns to identify leaders and followers. We demonstrate that the lead-lag relationships identified through this method are economically significant.

In this repository, we will familiarize ourselves with concepts used in financial lead-lag literature and implement some cutting-edge ideas in Python:

- Methods to mathematically define and extract pairwise lead-lag relationships between two time series.
- Implementing a pairwise lead-lag metric using the Lévy-area concept tied to signatures from rough path theory and constructing a lead-lag scoring matrix. This matrix is utilized for ranking assets, identifying assets from the most likely to be leaders to the most likely to be a followers.
- Building lead-lag directed networks for clustering and implementation of the lead-lag directed network Hermitian clustering.
- Implementation of Portfolio strategies to evaluate the economic significance of the discovered lead-lag relationships.

As the reference papers used in this repository present empirical results from the US equity market to demonstrate the economic significance of lead-lag portfolios using the Lévy area, we have chosen to apply the same approach to a different market—cryptocurrencies. This decision aims to provide a novel perspective rather than replicate the findings of the reference papers.

## __2. Methodology__

The methodology we followed in this repo, comprises three steps:

1. **Pairwise Lead-Lag Metric:** In the initial step, we apply a pairwise lead-lag metric to the returns of every possible pair of assets, generating scores for their lead-lag relationships. These pairwise scores are then used to construct a skew-symmetric matrix referred to as the lead-lag matrix, denoted as $S$.

2. **Global or Clustered Ranking:** In the second step, we utilize the lead-lag matrix to either globally rank assets from the most likely to be a leader to the most likely to be a follower or apply a clustering algorithm to the corresponding directed network of the lead-lag matrix, denoted as $G$. The goal is to identify clusterings with high pairwise flow imbalance between clusters. This process allows us to identify a set of leaders and a set of followers for use in the next step.

3. **Portfolio Construction and Evaluation:** After identifying sets of leaders and followers, we use the sign of the previous return of the set of leaders as a buy/sell signal to construct a portfolio. This allows us to evaluate the economic significance of the discovered lead-lag relationships.

## __3. Pairwise Metrics of Lead–Lag Relationship__

In a complex, non-linear system such as financial markets, determining a suitable way to define a metric to capture lead–lag relationships is challenging. Here we present some options.

### __3.1. Lead–Lag Metrics Based on a Functional of the Cross‑Correlation__

A commonly used approach to defining a lead–lag metric is to use a functional of a sample cross-correlation function ($CCF$) between two time series. The general form of a sample cross-correlation function between time series $i$ and $j$ evaluated at lag $l ∈ ℤ$ is given by:
\begin{align*}
CCF^{ij}(l) &= \text{corr}(Y^{i}_{t-l}, Y^{j}_t)
\end{align*}
where $corr$ denotes a choice of sample correlation function. There are several choices for the sample correlation function, i.e., $corr$, where the __Pearson linear correlation__ and __mutual Information__ based on discretized time series values are the most common. Pearson correlation can detect linear dependencies, while mutual information can detect general non-linear dependencies. Non-linear sample correlation functions, such as __Kendall rank correlation__ and __distance correlation__, are also available. Kendall rank correlation can detect monotonic non-linear dependencies, while distance correlation can detect general non-linear dependencies. The corresponding lead–lag metric, a measure of the extent to which $i$ leads $j$, is then obtained by:
\begin{align*}
S_{ij} = \text{$F$}(CCF^{ij})
\end{align*}
where $F$ is a suitable functional. Two common choices for the functional $F$ are as follows:

**1. CCF-Lag1:** Computes the difference of the cross-correlation function at lag ∈ {−1, 1}:
\begin{align*}
S_{ij} = CCF^{ij}(1) - CCF^{ij}(-1)
\end{align*}
**2. CCF-AUC:** Computes the signed normalized area under the curve (AUC) of the cross-correlation function:
\begin{align*}
S_{ij} = \frac{{(I(i, j) - I(j, i)) \cdot \max(I(i, j), I(j, i))}}{{I(i, j) + I(j, i)}}
\end{align*}

\begin{align*}
I(i, j) = \sum_{l=1}^{L} |corr(Y^{i}_{t-l}, Y^{j}_t)|
\end{align*}

These pairwise lead-lag metrics are not covered in this repository as they are straightforward to implement. Furthermore, using a functional of the cross-correlation function relies on the user to specify the choice of functional, which is not always obvious. In particular, it is difficult to gauge the number of lags to incorporate into our lead–lag metric a priori.

### __3.2 Lead–Lag Metric Based on Signatures__

As mentioned, the approach of using a functional of the cross-correlation function relies on the user to specify the choice of functional, and it is also difficult to gauge the number of lags to incorporate into the lead–lag metric a priori. An alternative approach draws on the idea of signatures from **rough path theory** to construct a pairwise lead–lag metric.

The signature of a continuous path with finite 1-variation $X ∶ [a, b] → R^d$, denoted by $S(X)_{a,b}$, is the collection of all the iterated integrals of $X$, namely $S(X)_{a,b} = (1, S(X)^{1}_{a,b}, … , S(X)^{d}_{a,b}, S(X)^{1,1}_{a,b}, S(X)^{1,2}_{a,b}, …)$, where the iterated integrals are given by:
\begin{align*} 
S(X)_{a,t}^{i_1, \ldots, i_k} = \int_{a<t_k < t} \ldots \int_{a<t_1 < t_2} dX^{i_1}_{t_1} \ldots dX^{i_k}_{t_k}
\end{align*}

The signatures-based pairwise measure of the lead–lag relation between two assets $i$ and $j$ over the time period $[t − m, t]$, which is called Lévy-area, is given by:

\begin{align*} 
S_{ij}(t - m, t) = \iint_{t-m<u<v<t} (dX^i(u)dX^j(v) - dX^j(u)dX^i(v)) 
\end{align*}

This represents the difference in the cross-terms of the second level of the time series signature. Theoretical results in rough path theory have established that a signature is essentially unique to the path it describes, and the truncated signature (i.e., the lower-order terms) can efficiently describe the path. The signature lead–lag metric is positive and grows larger whenever increases (or decreases) in $X^{i}$ are followed by increases (or decreases) in $X^{j}$. If the relative moves of $X^{i}$ and $X^{j}$ are in opposite directions, then the signature lead–lag measure is negative. It's worth noting that a downside of this method is that it is not able to distinguish between:

1. $i → j$ with negative association,
2. $i ← j$ with positive association.

As a result, we do not expect the method to perform well when there is signifcant negative association in the lead–lag data generating process.


We will use this metric in this repository to identify lead-lag relationships. When analyzing price data observed at discrete time points, we transform the data stream into a piecewise linear continuous path and calculate the second-order signatures. From this, we can calculate the lead–lag relation using the difference in second-order signature cross-terms. In practice, when comparing the signature lead–lag metrics across different pairs of time series, we recommend normalizing the price data before computing the lead–lag metric. This is because the absolute value of the metric increases with the volatility of the underlying price series.

## **4. Ranking Assets Using Lead-Lag Matrix, $S$**

Let $S_{ij}$ denote the user-defined lead–lag metric that quantifies how much the time series variable $i$ leads $j$. The value $S_{ij}$ can be positive or negative and satisfies $S_{ij} = -S_{ji}$. This is the lead-lag matrix $S$, and we can use various algorithms to rank the time series from the most likely to be a leader to the most likely to be a follower, globally. The simplest and most effective method for this ranking is to compute the **mean of each row of the $S$ matrix and order the rows according to their mean, from highest to lowest**. A priori, time series in rows with high means are more likely to be leaders, and those in rows with low means are more likely to be followers.

Alternative methods include **SpringRank**, **Serial Ranking**, and **SyncRank**. In synthetic data simulations, conducted in one of the reference papers, the authors test the ability of these ranking algorithms to identify lead-lag relationships with various levels of noise and observe that all three alternative ranking methods mentioned above can detect lead-lag relationships. In particular, SpringRank produces very similar rankings to those by the ranking by row average; on the other hand, Serial Ranking and SyncRank are more sensitive to the level of noise and the size of the lag than SpringRank and ranking by row average. Their observations in synthetic data simulation suggest that ranking by row average is the best choice because of its interpretability and performance. Thus, we will leverage row average method to sort assets in this repository.

## __5. Clustering Lead–Lag Directed Networks__

Using the $S$ matrix, we can create a directed network in which the nodes represent the time series, and edges represent the direction and intensity of the lead-lag relationships. However, since the $S$ matrix satisfies $S_{ij} = -S_{ji}$, the relationships between all pairs of time series must be encoded by the asymmetric matrix $A_{ij} = \max(S_{ij}, 0)$ to build a weighted and directed network $G$, where each node corresponds to a time series variable, and the adjacency matrix is $A$. There exist suitable algorithms for the detection of pairs of lead–lag clusters with high pairwise imbalance within this network. Some of the options for clustering directed networks are **Naive Symmetrization Clustering**, **Bibliometric Symmetrization Clustering**, **DI-SIM Co-Clustering**, and **Hermitian Clustering**. Since we aim to extract pairs of clusters with high flow imbalance, the Hermitian clustering algorithm is suited to our setting of clustering lead–lag networks as it is more robust than the other methods to various noise types and levels, as discussed in one of the reference papers. Therefore, we will implement and leverage this clustering algorithm in this repo.

## __6. Lead-Lag Portfolio Construction__

After identifying a set of assets that are most likely to be leaders and a set of assets that are most likely to be followers, either globally or using clustering of the directed networks, we should build a portfolio to demonstrate that the discovered relationships are economically significant. In this repo, we will cover three portfolios named Global, Clustered, and Global-Clustered portfolios. All of these portfolios are market-neutral portfolios designed to generate returns that are independent of the overall direction of the market. The goal of a market-neutral portfolio is to achieve a zero beta, meaning that the portfolio's performance is not significantly affected by movements in the broader market. We will discuss these portfolios in the relevant notebook, but it is important to emphasize that the goal of these portfolios is not to be used as investment strategies in the form they are presented in this repo, especially in the cryptocurrency market where there is still no tradeable index as a proxy for the market. This presents execution cost-related problems that degrade the performance of the portfolio, unlike markets such as the US equity market. So, do not rely on these portfolios in the cryptocurrency market in the form that they are presented in this repo. As we mentioned multiple times throughout this repo, they are just tools to show the economic significance of discovered latent lead-lag relationships.

## __7. Summary__

In the upcoming notebooks, we will delve into more details of the Lévy-area as the pairwise lead-lag metric of choice, Hermitian clustering of the directed lead-lag networks, and the construction and performance evaluation of the three portfolio types with their Python implementations.